In [1]:
from confluent_kafka import Producer
import json


In [2]:
p = Producer({'bootstrap.servers': 'kafka-1:19092,kafka-2:19093'})


In [3]:
def delivery_report(err, msg):
    """ Called once for each message produced to indicate delivery result.
        Triggered by poll() or flush(). """
    if err is not None:
        print('Message delivery failed: {}'.format(err))
    else:
        print('Message delivered Timestamp:{timestamp} to {} [{}] key:{}'.format(msg.topic(), msg.partition(), msg.key(),timestamp=msg.timestamp()))


In [4]:
lastone = ""

def WritePM10(boxJsonString):
    box = json.loads(boxJsonString)
    
    onlyNew= False
    global lastone
    lastRead = lastone
    for s in box['sensors']:   
      #  print("sensor tiles:{}".format(s['title']))
        if s['title'] == "PM10":
            isNewValue= not lastRead or (
                lastRead['lastMeasurement']['createdAt'] != 
                s['lastMeasurement']['createdAt'])
            if(not onlyNew or isNewValue):
              #  printTemperatureFromBox(s)
                lastone = s                
                p.poll(0)
                # Asynchronously produce a message, the delivery report callback
                # will be triggered from poll() above, or flush() below, when the message has
                # been successfully delivered or failed permanently.
                data =  s['lastMeasurement']['value']
                p.produce("clean-latest", key=box["_id"], value = data.encode('utf-8'), callback=delivery_report)

                # Wait for any outstanding messages to be delivered and delivery report
                # callbacks to be triggered.
                p.flush()


In [5]:
#%pip install -U avro-python3

In [6]:
# test
secondmessage= json.load('second_5b8449037c519100190fc72cTest.json')
firstmessage= json.load('first_5b8449037c519100190fc72cTest-Copy1.json')

AttributeError: 'str' object has no attribute 'read'

In [6]:
topicName ='raw'
from confluent_kafka import Consumer, KafkaError

c = Consumer({
    'bootstrap.servers': 'kafka-1:19092,kafka-2:19093',
    'group.id': 'test-consumer-group',
    'default.topic.config': {
        'auto.offset.reset': 'largest'
    }
})

c.subscribe([topicName])

In [7]:
#single run for testing
# todo fix avro schema
def SingleRun():
    go_on = True

    while go_on:
        msg = c.poll(1.0)

        if msg is None:
            continue
        if msg.error():
            if msg.error().code() == KafkaError._PARTITION_EOF:
                continue
            else:
                print(msg.error())
                break

        print('Received message: {}'.format(msg.value().decode('utf-8')))
        print('Received message: {}'.format(msg.key().decode('utf-8')))
        #WritePM10(msg.value().decode('utf-8'))
        go_on = False
        if msg.value().decode('utf-8') == "STOP":
            go_on = False

    c.close()

SingleRun()

Received message: [{"value":"8.40","location":[7.585112,47.555994],"createdAt":"2020-05-20T09:07:45.266Z"},{"value":"7.07","location":[7.585112,47.555994],"createdAt":"2020-05-20T09:02:40.975Z"},{"value":"7.65","location":[7.585112,47.555994],"createdAt":"2020-05-20T08:57:33.770Z"},{"value":"8.48","location":[7.585112,47.555994],"createdAt":"2020-05-20T08:52:29.516Z"},{"value":"12.27","location":[7.585112,47.555994],"createdAt":"2020-05-20T08:47:24.192Z"},{"value":"12.52","location":[7.585112,47.555994],"createdAt":"2020-05-20T08:42:13.578Z"},{"value":"13.50","location":[7.585112,47.555994],"createdAt":"2020-05-20T08:36:39.834Z"},{"value":"6.85","location":[7.585112,47.555994],"createdAt":"2020-05-20T08:31:03.598Z"},{"value":"8.88","location":[7.585112,47.555994],"createdAt":"2020-05-20T08:25:43.541Z"},{"value":"8.98","location":[7.585112,47.555994],"createdAt":"2020-05-20T08:20:33.274Z"},{"value":"11.20","location":[7.585112,47.555994],"createdAt":"2020-05-20T08:15:22.942Z"},{"value":

In [ ]:


#go_on = True
#
#while go_on:
#    msg = c.poll(1.0)
#
#    if msg is None:
#        continue
#    if msg.error():
#        if msg.error().code() == KafkaError._PARTITION_EOF:
#            continue
#        else:
#            print(msg.error())
#            break
#
#   # print('Received message: {}'.format(msg.value().decode('utf-8')))
#    WritePM10(msg.value().decode('utf-8'))
#    if msg.value().decode('utf-8') == "STOP":
#        go_on = False
#        
#c.close()

In [ ]:
go_on = False